In [40]:
!pip install matplotlib
!pip install pandas
!pip install scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import os
!pip install tdqm
from tqdm.notebook import tqdm
!pip install ipywidgets
!pip install eeglib
from eeglib.helpers import EDFHelper
from sklearn.decomposition import PCA
!pip install node2vec
!pip install networkx
import networkx as nx
from node2vec import Node2Vec



In [41]:
data_path = "data"
# find all files in the data directory
files = os.listdir(data_path)
files = sorted(files)
count = 0
stressed = []
not_stressed = []

# x is a pd dataframe with the features

# gt is a list of booleans, true if stressed, false if not
W = 250
overlap = 250
dataset=[]
S = 36
for t in tqdm(range(0,S)):
    unstressed_file = "Subject"+str(t).rjust(2, '0')+"_1.edf"
    stressed_file = "Subject"+str(t).rjust(2, '0')+"_2.edf"
    # read the edf file
    gt=[]
    x=[]
    is_stressed = False
    all_files = []
    vectors = []
    for file in [unstressed_file, stressed_file]:
        helper= EDFHelper("data/"+str(file), sampleRate=500, lowpass=30, highpass=0.5, windowSize=W)
        matrices = []
        for eeg in helper:
            # for this particular window
            dat = eeg.getChannel()
            # create N by N matrix

            l = len(dat)
            mat = [[0 for i in range(l)] for j in range(l)]
            for i in range(l):
                for j in range(l):
                    # calculate pearson's correlation between dat[i] and dat [j]
                    mat[i][j] = np.corrcoef(dat[i], dat[j])[0][1]
            matrices.append(mat)
            gt.append(is_stressed)

        print(len(matrices))

        # convert the list of matrices to a list of features
        nw = len(matrices)
        B = [[0 for i in range(nw)] for j in range(nw)]
        #create a new nx graph
        G = nx.Graph()
        # create nodes in the graph
        for i in range(nw):
            G.add_node(i)
        for i in range(nw):
            for j in range(nw):
                if i != j:
                    # calculate the similarity between matrices[i] and matrices[j]
                    G.add_edge(i, j, weight=np.linalg.norm(np.array(matrices[i]) - np.array(matrices[j])))
        # create a node2vec object
        node2vec = Node2Vec(G, dimensions=30, walk_length=16, num_walks=100, workers=8, quiet=1)
        # train node2vec
        model = node2vec.fit()
        # get the vectors
        vecs=model.wv
        # convert the vectors to a list
        vecs = [vecs[i] for i in range(nw)]
        vectors.extend(vecs)

        # append the list to x
        # using node2vec, convert B[i][j] to vectors. This is the feature vector for the window
        # append the feature vector to x
        is_stressed = True


    dataset.append((vectors, gt))




  0%|          | 0/36 [00:00<?, ?it/s]

364
124
364
124
364
124
364
124
340
124
364
124
364
124
364
124
364
124
364
124
376
124
364
124
364
124
364


KeyboardInterrupt: 

In [42]:

accuracies = []
for person in dataset:
    x=person[0]
    gt=person[1]
    x=pd.DataFrame(x)
    # convert x from a list to a pd dataframe

    # convert gt from a list to a pd dataframe
    x_train, x_test, y_train, y_test = train_test_split(x, gt, test_size=0.5, random_state=42)
    # convert the dataframes to numpy arrays
    # create the vectorizer
    # fit the vectorizer to the training data
    # scale the training data
    '''
    pca_30 = PCA(n_components=30, random_state=69)
    x_train = pca_30.fit_transform(x_train)
    x_test = pca_30.transform(x_test)
    '''

    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(y_test, y_pred)

    accuracies.append(accuracy_score(y_test, y_pred))

print(accuracies)
print("Average Accuracy:", np.mean(accuracies))

[False, False, True, False, True, True, False, False, False, False, False, False, True, True, False, True, True, False, False, True, False, False, False, False, True, False, True, False, False, True, False, True, False, False, True, False, False, True, False, False, True, False, True, True, True, False, True, False, True, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, True, False, True, True, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, False, True, False, False, True, False, False, False, False, False, False, False, True, True, True, False, False, False, False, False, True, False, True, True, True, False, False, True, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, True, F